# Predicting house prices using k-nearest neighbors regression
In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [2]:
sales = turicreate.SFrame('kc_house_data_small.csv')

Finished parsing file /home/newuser/venv/course2/kc_house_data_small.csv

Parsing completed. Parsed 100 lines in 0.096854 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/newuser/venv/course2/kc_house_data_small.csv

Parsing completed. Parsed 8703 lines in 0.077669 secs.

# Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead

In [4]:
def get_numpy_data(data,feature,output):
    data['constant'] = 1
    features = ['constant'] + feature
    feature_sframe = data[features]
    feature_matrix = feature_sframe.to_numpy()
    output_sarray = data[output]
    output_array = output_sarray.to_numpy()
    return (feature_matrix,output_array)

We will also need the `normalize_features()` function from Week 5 that normalizes all feature columns to unit norm. Paste this function below.

In [5]:
def normalize_features(feature_matrix):
    norm = np.linalg.norm(feature_matrix,axis=0)
    normalized_features = (feature_matrix / norm)
    return (normalized_features,norm)

# Split data into training, test, and validation sets

In [6]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

# Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [7]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [8]:
normalized_features_train, norms = normalize_features(features_train) # normalize training set features (columns)
normalized_features_test = features_test / norms # normalize test set by training set norms
normalized_features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [9]:
def compute_distances(features_train_matrix, query_house_vector):
    
    distances = np.sqrt(np.sum((features_train_matrix - query_house_vector)**2, axis=1))
    
    return distances

distances_house10 = compute_distances(normalized_features_train[0:10],normalized_features_test[0])
distances_house10

array([0.06027471, 0.08546881, 0.06149946, 0.05340274, 0.05844484,
       0.05987922, 0.0546314 , 0.05543108, 0.05238363, 0.05972359])

Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 10th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once.

# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [10]:
for i, distance in enumerate(distances_house10):
    if distance == min(distances_house10):
        print("house order number of closest house in training set to query house: {}".format(i))
        print("distance: {}".format(distance))


house order number of closest house in training set to query house: 8
distance: 0.052383627840220305


*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [11]:
for i in range(3):
    print (features_train[i]-features_test[0])
    # should print 3 vectors of length 18

[ 0.00e+00 -1.00e+00 -2.00e+00 -1.77e+03  6.50e+02 -1.00e+00  0.00e+00
 -3.00e+00  0.00e+00 -2.00e+00 -8.00e+02 -9.70e+02 -2.40e+01  0.00e+00
 -6.02e-02  1.18e-01 -8.00e+02  1.65e+03]
[ 0.000e+00 -1.000e+00 -7.500e-01 -3.800e+02  2.242e+03  0.000e+00
  0.000e+00 -3.000e+00  0.000e+00 -2.000e+00  1.900e+02 -5.700e+02
 -2.800e+01  1.991e+03  1.496e-01  5.600e-02 -4.500e+02  3.639e+03]
[ 0.000e+00 -2.000e+00 -2.000e+00 -2.180e+03  5.000e+03 -1.000e+00
  0.000e+00 -3.000e+00  0.000e+00 -3.000e+00 -1.210e+03 -9.700e+02
 -4.600e+01  0.000e+00  1.665e-01  1.420e-01  5.800e+02  4.062e+03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [12]:
print (features_train[0:3] - features_test[0])

[[ 0.000e+00 -1.000e+00 -2.000e+00 -1.770e+03  6.500e+02 -1.000e+00
   0.000e+00 -3.000e+00  0.000e+00 -2.000e+00 -8.000e+02 -9.700e+02
  -2.400e+01  0.000e+00 -6.020e-02  1.180e-01 -8.000e+02  1.650e+03]
 [ 0.000e+00 -1.000e+00 -7.500e-01 -3.800e+02  2.242e+03  0.000e+00
   0.000e+00 -3.000e+00  0.000e+00 -2.000e+00  1.900e+02 -5.700e+02
  -2.800e+01  1.991e+03  1.496e-01  5.600e-02 -4.500e+02  3.639e+03]
 [ 0.000e+00 -2.000e+00 -2.000e+00 -2.180e+03  5.000e+03 -1.000e+00
   0.000e+00 -3.000e+00  0.000e+00 -3.000e+00 -1.210e+03 -9.700e+02
  -4.600e+01  0.000e+00  1.665e-01  1.420e-01  5.800e+02  4.062e+03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [13]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print (results[0] - (features_train[0]-features_test[0]))
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print (results[1] - (features_train[1]-features_test[0]))
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print (results[2] - (features_train[2]-features_test[0]))
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [14]:
diff = normalized_features_train - normalized_features_test[0]
diff[-1]

array([ 0.00000000e+00, -3.87821276e-03, -3.01245476e-03, -8.46807236e-03,
       -1.24208957e-03,  8.52950206e-03,  0.00000000e+00, -5.10236549e-02,
        0.00000000e+00, -1.73816863e-03, -3.09564484e-03, -2.48168183e-02,
        2.04695248e-04,  0.00000000e+00,  3.61719513e-05,  3.19188881e-06,
       -3.92203156e-03, -1.01041218e-03])

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [15]:
print (diff[-1].sum()) # sum of the feature differences between the query and last training house
# should print -0.0934339605842

-0.09343399874654643


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [16]:
print (np.sum(diff**2, axis=1)[15]) # take sum of squares across each row, and print the 16th sum
print (np.sum(diff[15]**2)) # print the sum of squares for the 16th row -- should be same as above

0.0033070590284564457
0.0033070590284564453


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [17]:
distances = compute_distances(normalized_features_train,normalized_features_test[0])

To test the code above, run the following cell, which should output a value 0.0237082324496:

In [18]:
print (distances[100]) # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496

0.023708232416678195


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [19]:
query_house2 = compute_distances(normalized_features_train,normalized_features_test[2])

In [20]:
for i,distance in enumerate(query_house2):
    if distance == min(query_house2):
        print("house order number of closest house in training set to query house: {}".format(i))
        print("distance: {}".format(distance))


house order number of closest house in training set to query house: 382
distance: 0.0028604955575117085


# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [21]:
def nearest_neighbor(query_house):
    for i,distance in enumerate(query_house):
        if distance == min(query_house):
            print("house order number of closest house in training set to query house: {}".format(i))
            print("distance: {}".format(distance))
    return('-------------')

In [22]:
k_nearest = compute_distances(normalized_features_train[5:10],normalized_features_test[1])

In [23]:
nearest_neighbor(k_nearest)

house order number of closest house in training set to query house: 4
distance: 0.006308009384481238


'-------------'

In [24]:
def fetch_knn(k, features_train_matrix, feature_vector_query_house):
    
    distances_vector_all = compute_distances(features_train_matrix, feature_vector_query_house)
    
    indices_closest_train_houses = np.argsort(distances_vector_all)[0:k]
    
    return indices_closest_train_houses

In [25]:
fetch_knn(10,normalized_features_train[5:10],normalized_features_test[1])

array([4, 0, 1, 3, 2])

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [26]:
fetch_knn(4,normalized_features_test[0:4],normalized_features_test[2])

array([2, 3, 1, 0])

## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

In [27]:
def predict_price_query_house(k, features_train_matrix, output_train, feature_vector_query_house):
    
    indices_closest_train_houses = fetch_knn(k, features_train_matrix, feature_vector_query_house)
    
    predicted_price_query_house = np.mean(output_train[indices_closest_train_houses])
    
    return predicted_price_query_house

*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

In [28]:
predict_price_query_house(5,normalized_features_train[6:10],output_train,normalized_features_test[2])

385975.0

In [29]:
predict_price_query_house(10,normalized_features_train,output_train,normalized_features_test[0])

881300.0

Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix. See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [30]:
def predict_value_all_houses(k, features_train_matrix, output_train, feature_test):
    predicted_prices = []
    
    for i in range(features_test.shape[0]):
        predicted_price = predict_price_query_house(k, features_train_matrix, output_train, features_test[i])
    
        predicted_prices.append(predicted_price)
    
    return predicted_prices


*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [31]:
x = [] 
for i in range(normalized_features_test.shape[0]):
        predicted_price = predict_price_query_house(10, normalized_features_train, output_train, normalized_features_test[i])
        x.append(predicted_price)
        print(predicted_price)

881300.0
431860.0
460595.0
430200.0
766750.0
667420.0
350032.0
512800.7
484000.0
457235.0
487160.0
403770.0
423762.6
287847.5
734550.0
472950.0
589448.3
263150.0
509200.0
488090.0
501775.0
956205.0
753170.0
738330.0
717190.0
401863.8
278065.0
354690.0
412045.0
403600.0
636390.0
423955.0
423298.1
1146910.0
251890.0
391945.0
449940.0
843499.0
1088550.0
392590.0
353000.0
543050.0
266705.0
2280500.0
879250.0
561900.0
512420.0
383399.0
480350.0
417000.0
479505.0
390995.0
506400.7
530345.0
357345.0
382250.0
523550.0
387532.2
422860.0
262200.0
441185.0
492935.0
389415.0
565902.2
353695.0
382000.0
423800.0
354390.0
290945.0
583292.5
444783.8
851975.0
260165.0
444295.0
223240.0
492408.3
411328.3
506645.0
378000.0
272985.0
302627.5
484045.0
411250.0
512200.0
470900.0
425690.0
615600.0
388055.0
617157.2
292150.0
1038995.0
507095.0
744957.5
1814650.0
477500.0
530009.7
215195.0
447950.0
797300.0
458845.0
564722.6
372470.0
372185.0
2541730.0
383355.0
644845.0
470900.0
632300.0
524895.0
862305.0
3682

669645.0
2381730.0
391600.1
440908.3
604035.0
604750.0
306540.0
274820.3
489825.0
1312199.0
294400.0
1045200.0
278374.0
430970.0
681300.0
562808.8
393595.0
672644.0
311857.5
522350.0
486395.0
337949.9
301338.8
352043.4
655325.0
450400.0
623474.2
520895.0
380969.0
755380.0
539584.0
354590.0
345543.8
668950.0
316150.0
556695.0
406080.0
323370.0
348250.0
504800.0
376166.7
941763.8
960600.0
282975.0
307395.0
387495.0
782305.1
508330.0
268670.0
911465.0
340745.0
338095.0
353595.0
457190.0
782045.0
725090.0
581290.0
386500.9
406404.7
530734.3
452880.3
1108699.5
455550.0
506750.0
483805.0
279860.0
404471.9
715540.0
992580.0
502290.0
298827.5
313477.5
244990.0
217170.0
1341950.0
1169280.0
430085.0
585815.3
716500.0
462939.9
450090.0
431350.0
305950.0
457500.0
513331.2
366740.0
423345.0
391295.0
517159.0
766788.0
1471850.0
900794.5
488675.0
1527695.4
529650.0
339625.0
429885.0
786190.0
502090.0
1294825.0
342400.0
363269.0
395445.0
313065.0
633325.0
456945.0
470200.0
278110.0
359145.0
402875.0
4

In [34]:
lowest_predict_value = min(x)

In [35]:
lowest_predict_value

181588.8

## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

In [36]:
def compute_rss(predictions,output):
    error = predictions - output
    RSS = pow(error,2)
    return(RSS)

(Depending on your computing environment, this computation may take 10-15 minutes.)

In [53]:
rss_all = []
for i in range(normalized_features_valid.shape[0]):
    for k in range(1,16):
         predictions_valid = predict_price_query_house(k, normalized_features_train, output_train, normalized_features_valid[i])
    
         RSS = compute_rss(predictions_valid,output_valid)
    
         rss_all.append(RSS)



In [56]:
rss = []
for k in range(1,16):
         predictions_valid = predict_price_query_house(k, normalized_features_train, output_train, normalized_features_valid[for i in range(normalized_features_valid.shape[0])])
    
         RSS = compute_rss(predictions_valid,output_valid)
    
         rss.append(RSS)

SyntaxError: invalid syntax (<ipython-input-56-81869dd10f12>, line 3)

In [54]:
len(normalized_features_valid)

1435

In [55]:
len(rss_all)

21525

To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [40]:
for i,val in enumerate(rss_all):
    
    if val == min(rss_all):
        print("k that produced the lowest RSS on validation set: k = {}".format(i))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

kvals = range(1, 16)
plt.plot(kvals, rss_all,'bo-')

***QUIZ QUESTION ***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.